In [4]:
import numpy as np
import sys 

assert sys.version_info[:3] >= (3, 6, 0), "Make sure you have Python 3.6 installed!"

## The Multi-armed bandit problem
Imagine you're faced with a number of slot machines (also called 'bandits'), each with a lever ('arm') to pull. Upon pulling a particular lever, it will give a random reward from an unknown distribution particular to that lever. The goal of the multi-armed bandit (MAB) problem, is to maximise your total reward given that you are allowed to pull levers a fixed number of times total (this is called your 'budget').

A basic strategy might be to spend some of your budget pulling different levers to get an idea of which levers give the most reward ('exploration'). After this, you may choose increasingly often pull the lever that you expect gives the most reward ('exploitation'). The question then is: how much exploration and how much exploitation makes the optimal strategy? This 'exploration-exploitation trade-off' is a classic feature of reinforcement learning problems: we have to interact with the environment to gather data, and we must choose an optimal way of interacting on the fly. 

This notebook provides a MAB environment to interact with. Spend some time pulling levers to get a feeling for the problem (see cells below).

In [5]:
# A bandit gives a random reward from a particular Gaussian distribution.
class Bandit:
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std
        
    def sample(self):
        return np.random.normal(self.mean, self.std)

In [6]:
class MultiArmedBandit:
    def __init__(self, num_arms=10, means=None, stds=None):
        if means is None:
            self.means = np.random.uniform(0, 5, num_arms)
        else:
            self.means = means
        if stds is None:
            self.stds = np.random.uniform(0, 3, num_arms)
        else:
            self.stds = stds 
        self.bandits = [Bandit(mean, std) for mean, std in zip(self.means, self.stds)]
        self.arms_pulled = np.zeros(num_arms, dtype=int)
        self.arms_rewards = np.zeros(num_arms)
        self.num_arms = num_arms
        
    def reset(self):
        self.__init__(self.num_arms, self.means, self.stds)
        
    def sample(self, i):
        reward = self.bandits[i].sample()
        self.arms_pulled[i] += 1
        self.arms_rewards[i] += reward
        return reward
    
    def get_state(self):
        return self.arms_rewards, self.arms_pulled

### Get a feeling
Play around with the arms for a minute by running the cell below.

In [7]:
# Simple example interaction 
num_arms = 4
mab = MultiArmedBandit(num_arms)
for _ in range(10):
    arm = int(input(f"Choose an arm to pull [0-{num_arms-1}]: "))
    assert arm >=0 and arm < num_arms, f"Arm must be an integer in the interval [0, {num_arms - 1}] inclusive."
    print(" Reward: {:.3f}".format(mab.sample(arm)))

Choose an arm to pull [0-3]: 0
 Reward: 0.260
Choose an arm to pull [0-3]: 0
 Reward: 5.164
Choose an arm to pull [0-3]: 0
 Reward: -0.693
Choose an arm to pull [0-3]: 1
 Reward: 3.065
Choose an arm to pull [0-3]: 1
 Reward: 3.912
Choose an arm to pull [0-3]: 1
 Reward: 3.776
Choose an arm to pull [0-3]: 2
 Reward: 2.222
Choose an arm to pull [0-3]: 2
 Reward: 0.463
Choose an arm to pull [0-3]: 2
 Reward: 1.390
Choose an arm to pull [0-3]: 3
 Reward: -0.003


### Example estimation
Below is an example interaction that tries to estimate the best arm of a 10-armed bandit from 100 samples.

In [8]:
# Example interaction
num_arms = 10
mab = MultiArmedBandit(num_arms)
# Sample 100 random arms
for _ in range(100):
    action = np.random.choice(num_arms)
    reward = mab.sample(action)

# Get how many times arms were pulled and how much total reward was generated by those arms.
# Together these arrays represent the state of the MAB.
state = mab.get_state()
arms_rewards, arms_pulled = state
# Get average reward per arm
arms_average_reward = arms_rewards / arms_pulled

# Inspect results
best_arm, best_reward = -1, -10e3
for i, average_reward in enumerate(arms_average_reward):
    print('Arm {} yielded average reward: {:.3f}'.format(i, average_reward))
    if average_reward > best_reward:
        best_reward = average_reward
        best_arm = i
print('\nWe think the best arm is arm {}'.format(best_arm))

Arm 0 yielded average reward: 3.066
Arm 1 yielded average reward: 0.692
Arm 2 yielded average reward: 4.912
Arm 3 yielded average reward: 1.039
Arm 4 yielded average reward: 2.908
Arm 5 yielded average reward: 3.852
Arm 6 yielded average reward: 3.081
Arm 7 yielded average reward: 1.854
Arm 8 yielded average reward: 0.104
Arm 9 yielded average reward: 1.051

We think the best arm is arm 2


The goal of this exercise is to get a feeling for this MAB problem. In order to do this, you're tasked with writing a strategy (policy) that maximises the expected reward, or - equivalently - mimises the expected regret (where the expectation is taken over multiple simulations where a new MAB is instantiated each time). Regret of a policy is defined here as: expected optimal reward - expected obtained reward. That is: it is the difference between how much reward an oracle that knows the optimal lever to pull would have obtained, and the reward the implemented policy obtains.

Below a 'simulate_policies' function is provided that calculates this expected regret, given a policy (or list of policies for fair comparison of policies). A policy is a function that takes as input a state (in this case the tuple (arms_pulled, arms_rewards)), and outputs an action (in this case an integer in the interval [0, num_arms - 1] inclusive). Two example policies are provided: random, and a policy that starts random (we call this a burn-in period) and then proceeds to pull the lever it thinks is best based on the statistics gathered during the burn-in. 

This last policy is a very naive way to dealing with the exploration-exploitation trade-off: first we explore for a set number of samples, then we exploit for the rest of the budget. See if you can write a policy that improves over this one. Note that to really evaluate this well we need to run simulations very often, which might be infeasible given the time we want to spend on this notebook. If you get something that does approximately as well as this policy, consider it a success.

In [9]:
def episode(policy, budget, mab=None, num_arms=10):
    """
    Function used to simulate an episode. Takes as input a policy, and outputs regret.
    
    Args:
        policy (callable): A function that takes as input a state tuple (arms_rewards, arms_pulled)
            and outputs an integer in the interval [0, num_arms - 1] inclusive that represents the
            action to take.
        budget: number of samples to draw before an episode terminates. 
        
    Returns:
        average_regret (float): average regret over the episode.
    """
    if mab is None:
        mab = MultiArmedBandit(num_arms)
    optimal_reward = np.max(mab.means) * budget
    for _ in range(budget):
        state = mab.get_state()
        choice = policy(state)
        mab.sample(choice)
    total_reward = np.sum(mab.arms_rewards)
    regret = (optimal_reward - total_reward)
    return regret


def simulate_policies(policies, num_arms=10, budget=1000, num_simulations=100):
    """
    Args:
        policies (callable or list of callables): A list of functions that each take as input a state 
            tuple (arms_rewards, arms_pulled) and output an integer in the interval [0, num_arms - 1] 
            inclusive that represents the action to take.
        num_arms: number of arms on the MultiArmedBandit.
        budget: number of samples to draw before an episode terminates.
        num_simulations: number of episodes to average the results over.   
        
    Returns:
        expected_regrets (list or float): list of expected regrets corresponding to the policies. Float
            if a single policy was evaluated.
    """
    if not isinstance(policies, list):
        policies = [policies]
    average_regrets = np.zeros(len(policies))
    for _ in range(num_simulations):
        mab = MultiArmedBandit(num_arms)
        for i, policy in enumerate(policies):
            if i > 0:
                mab.reset()
            regret = episode(policy, budget, mab)
            average_regrets[i] += regret / num_simulations
            
    if len(average_regrets) == 1:
        return average_regrets[0]
    return list(average_regrets)


def random_policy(state):
    """
    Random policy.
    
    Args:
        state (tuple): a tuple (arms_rewards, arms_pulled) representing the state of the MAB.
            'arms_rewards' is a numpy array of length num_arms, that represents the total reward obtained
            from pulling arms. 'arms_pulled' is a numpy array of the same length that represents the 
            number of times a particular arm was pulled.
            
    Returns:
        action (int): integer in the interval [0, num_arms - 1] inclusive, representing the arm to pull.
    """
    arms_rewards, arms_pulled = state
    action = np.random.choice(len(arms_rewards))
    return action


def max_policy_with_burnin(state, burnin=100):
    """
    Policy that selects random levers during a burn-in (exploration), followed by 
    exploitation of the optimal lever according to the gathered statistics.

    Args:
        state (tuple): a tuple (arms_rewards, arms_pulled) representing the state of the MAB.
            'arms_rewards' is a numpy array of length num_arms, that represents the total reward obtained
            from pulling arms. 'arms_pulled' is a numpy array of the same length that represents the 
            number of times a particular arm was pulled.
            
    Returns:
        action (int): integer in the interval [0, num_arms - 1] inclusive, representing the arm to pull.
    """
    arms_rewards, arms_pulled = state
    if np.sum(arms_pulled) < burnin:
        action = np.random.choice(len(arms_rewards))
        return action
    average_arm_reward = arms_rewards / arms_pulled
    action = np.argmax(average_arm_reward)
    return action

policies = [random_policy, max_policy_with_burnin]
random_policy_regret, max_policy_regret = simulate_policies(policies)
print('Random policy regret: {:.2f}'.format(random_policy_regret))
print('Max policy regret: {:.2f}'.format(max_policy_regret))

Random policy regret: 2029.12
Max policy regret: 258.65


In [31]:
def optimistic_exploration(state):
    """
    Args:
        state (tuple): a tuple (arms_rewards, arms_pulled) representing the state of the MAB.
            'arms_rewards' is a numpy array of length num_arms, that represents the total reward obtained
            from pulling arms. 'arms_pulled' is a numpy array of the same length that represents the 
            number of times a particular arm was pulled.
            
    Returns:
        action (int): integer in the interval [0, num_arms - 1] inclusive, representing the arm to pull.
    """
    
    arms_rewards, arms_pulled = state
    T = np.sum(arms_pulled)
    c = 2000
    exploration_bonus = np.sqrt((c*np.log(T))/arms_pulled)
#     exploration_bonus = np.sqrt(1/arms_pulled)
#     exploration_bonus = 1/arms_pulled

#     exploration_bonus = c*np.sqrt((np.log(T))/arms_pulled)
    ucb = arms_rewards + exploration_bonus
    
    action = np.argmax(ucb)

    return action

policies = [optimistic_exploration, max_policy_with_burnin]
my_policy_regret, max_policy_regret = simulate_policies(policies)
print('My policy regret: {:.2f}'.format(my_policy_regret))
print('Max policy regret: {:.2f}'.format(max_policy_regret))

/home/david/miniconda3/envs/rl2019/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()
/home/david/miniconda3/envs/rl2019/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in sqrt
  app.launch_new_instance()
/home/david/miniconda3/envs/rl2019/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()
/home/david/miniconda3/envs/rl2019/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in true_divide
  app.launch_new_instance()


My policy regret: 224.57
Max policy regret: 253.54
